In [44]:
import pandas as pd
from utils.load_dataframe import load_dataframe

In [45]:
reviews = load_dataframe("order_reviews")
orders = load_dataframe("orders")

In [46]:
# Inspeção inicial: checar presença de mensagens ou títulos nos reviews
display(reviews["review_comment_message"].unique()[:5])
display(reviews["review_comment_title"].unique()[:5])

array(['Sem comentário', 'Recebi bem antes do prazo estipulado.',
       'Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa',
       'aparelho eficiente. no site a marca do aparelho esta impresso como 3desinfector e ao chegar esta com outro nome...atualizar com a marca correta uma vez que é o mesmo aparelho',
       'Mas um pouco ,travando...pelo valor ta Boa.\r\n'], dtype=object)

array(['Sem título', 'recomendo', 'Super recomendo',
       'Não chegou meu produto ', 'Ótimo'], dtype=object)

In [47]:
# Estatísticas descritivas das avaliações
avg_review_score = reviews["review_score"].mean().round(2)
display(f"Média das avaliações: {avg_review_score}")

median_review_score = reviews["review_score"].median()
display(f"Mediana das avaliações: {median_review_score}")

'Média das avaliações: 4.09'

'Mediana das avaliações: 5.0'

In [48]:
# Distribuição das notas (quantidade por score)
note_counts = reviews["review_score"].value_counts().sort_index().reset_index()

note_counts.columns = ["Nota", "Quantidade"]

note_counts

,Nota,Quantidade
0,1,11424
1,2,3151
2,3,8179
3,4,19142
4,5,57328


In [ ]:
# Correlação entre atrasos e avaliações dos clientes

# Calcula diferença entre entrega estimada e real (dias)
# delay_diff < 0 significa que o pedido atrasou
orders["delay_diff"] = (
    orders["order_estimated_delivery_date"] - orders["order_delivered_customer_date"]
).dt.days

orders["delay"] = orders["delay_diff"] < 0

# Junta pedidos com avaliações
order_review_join = orders.merge(reviews, on="order_id")

# Taxa de pedidos atrasados
total_orders = len(order_review_join)
delayed_orders = len(order_review_join[order_review_join['delay']])

delayed_orders_rate = round((delayed_orders / total_orders) * 100, 2)
display(f'Taxa de pedidos atrasados: {delayed_orders_rate}%')

# Média da nota de pedidos com e sem atrasos
comparison_rating_delayed = (
    order_review_join.groupby("delay")["review_score"]
    .mean()
    .reset_index()
    .round(2)
)

comparison_rating_delayed

'Taxa de pedidos atrasados: 7.76%'

,delay,review_score
0,False,4.21
1,True,2.57


# Correlação de clientes e pedidos atrasados

---

Avaliar o efeito dos atrasos na entrega para entender como a experiência logística impacta satisfação do cliente.

- Pedidos entregues **no prazo** têm satisfação média de **4.21**
- Pedidos **atrasados** caem para **2.57**

Isso representa uma queda de aproximadamente **39% na satisfação em média**.

Além disso, foram analisados *99224* pedidos avaliados, dos quais **7.76% apresentaram atrasos**. O atraso se confirma como um dos fatores mais críticos na redução das notas.

---

**Conclusão:**

Há uma queda significativa de satisfação quando ocorre atraso, indicando que melhorias no processo logístico podem gerar impacto imediato na percepção do cliente, aumentar as notas e reduzir reclamações.
